In [ ]:
import pandas
import seaborn
import nglview

In [ ]:
import proteinpile
import proteinpile.pile

spec = proteinpile.common.get_spec("specification.py")
pile = proteinpile.pile.Pile("manifest.csv", "intermediates/")
pile

In [ ]:
all_df = pandas.concat(
    [pile.manifest] + [
        pandas.DataFrame.from_records(pile.manifest[col].values, index=pile.manifest.index)
        for col in pile.manifest.columns
        if col.endswith("_dict")
    ],
    axis=1)
all_df["design"] = pile.get_designs(spec, names=all_df.index)
all_df

In [ ]:
df = all_df.loc[
    all_df[["af2_ca_rmsd_to_design", "omegafold_ca_rmsd_to_design"]].max(1) < 2
].copy()
df.shape

In [ ]:
selected_df = df.loc[
    (df.af2_ca_rmsd_to_design < 2) &
    (df.omegafold_ca_rmsd_to_design < 2) &
    (df.af2_motif_0_all_atom_rmsd < 3)
]
selected_df.shape

In [ ]:
nglview.show_prody(selected_df.iloc[2].design.get_structure("af2"))

In [ ]:
out = "selected.pml"
with open(out, "w") as fd:
    lines = spec.pymol_lines(selected_df.design.values)
    for line in lines:
        print(line)
        fd.write(line)
        fd.write("\n")
print("Wrote pymol script to", out)